In [1]:
import pickle
from util_fdp import *

/root/miniconda3/envs/myconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-15 07:43:58,949	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
mode='federated'
T_mode='samples'  # samples / rounds
gene_process = 'homo' # 'homo /hete' / 'hete_d'


In [5]:
cvg_case2 = load_pickle(f'output/case_{mode}_{T_mode}_{gene_process}_cvg.pkl')
mae_case2 = load_pickle(f'output/case_{mode}_{T_mode}_{gene_process}_mae.pkl')

print(cvg_case2)
print(mae_case2)

FileNotFoundError: [Errno 2] No such file or directory: 'output/case_federated_samples_homo_cvg.pkl'

In [14]:
# print(cvg_case2)
# print(mae_case2)

In [15]:
def generate_combined_table(data_ci, data_mae, caption="Results",
                            label="tab:combined"):
    # 自动获取所有n值并按数字顺序排序
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())), key=int)
    latex_code = f""
    for n in all_n:
        # 添加n标题行
        latex_code += f"\\multicolumn{{6}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"
        
        # 遍历所有tau和r组合
        for tau in sorted(data_ci[n].keys(), key=float):
            for r_val in data_ci[n][tau].keys():
                # 生成行内容
                row = f"{tau} & {r_val if isinstance(r_val, str) else r_val} "
                
                # 遍历所有E值
                for e in data_ci[n][tau][r_val].keys():
                    ci = data_ci[n][tau][r_val].get(e, "--")
                    mae = data_mae[n][tau][r_val].get(e, "--")
                    cell = f"& {ci:.3f}({mae:.4f}) " if isinstance(ci, float) and isinstance(mae, float) else "& -- "
                    row += cell
                
                latex_code += row + "\\\\\n"
            
            # 添加tau之间的分隔线
            latex_code += "\\hline\n"
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table}"""
    return latex_code

In [16]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open(f"output/tex_{mode}_{T_mode}_{gene_process}.tex", "w") as f:
    f.write(latex_output)

## For Global

In [13]:
mode='global'
gene_process = 'hete_d' # 'homo /hete' / 'hete_d'


In [14]:
cvg_case2 = load_pickle(f'output/case_{mode}_{gene_process}_cvg.pkl')
mae_case2 = load_pickle(f'output/case_{mode}_{gene_process}_mae.pkl')

print(cvg_case2)
print(mae_case2)

{10000: {0.5: {0.25: np.float64(0.971), 'hetero': np.float64(0.999), 0.9: np.float64(1.0)}}, 50000: {0.5: {0.25: np.float64(0.984), 'hetero': np.float64(1.0), 0.9: np.float64(1.0)}}}
{10000: {0.5: {0.25: np.float64(0.06744390042068414), 'hetero': np.float64(0.02855844939542787), 0.9: np.float64(0.01471459623144639)}}, 50000: {0.5: {0.25: np.float64(0.02076778282180595), 'hetero': np.float64(0.00767371188788854), 0.9: np.float64(0.003425308940953684)}}}


In [15]:
def generate_combined_table(data_ci, data_mae,
                            caption="Results",
                            label="tab:combined"):
    r"""
    生成 LaTeX 表格，行格式：
        tau  &  r  &  ci(mae)  \\
    适用于 data_ci[n][tau][r]  以及  data_mae[n][tau][r]  这种三层嵌套结构
    """

    # ── 收集并排序所有 T ───────────────────────────────────────────
    all_n = sorted(set(list(data_ci.keys()) + list(data_mae.keys())),
                   key=int)

    latex_code = (
        "\\begin{table}[htbp]\n"
        "\\centering\n"
        f"\\caption{{{caption}}}\n"
        f"\\label{{{label}}}\n"
        "\\begin{tabular}{lll}\n"   # 3 列：tau, r, ci(mae)
        "\\toprule\n"
        "τ & r & CI(MAE)\\\\\n"
        "\\midrule\n"
    )

    for n in all_n:
        # ★ 3 列 → \multicolumn{3}
        latex_code += f"\\multicolumn{{3}}{{c}}{{\\textbf{{T = {n}}}}} \\\\\n"
        latex_code += "\\midrule\n"

        # 遍历 tau
        for tau in sorted(data_ci[n].keys(), key=float):
            # 遍历 r
            for r_val in data_ci[n][tau].keys():
                ci  = data_ci[n][tau].get(r_val, "--")
                mae = data_mae[n][tau].get(r_val, "--")

                cell = (
                    f"{ci:.3f}({mae:.4f})"
                    if isinstance(ci, float) and isinstance(mae, float)
                    else "--"
                )
                row = (
                    f"{tau} & "
                    f"{r_val if isinstance(r_val, str) else r_val} & "
                    f"{cell} \\\\"
                )
                latex_code += row + "\n"

            # tau 之间分隔线
            latex_code += "\\hline\n"

    latex_code += (
        "\\bottomrule\n"
        "\\end{tabular}\n"
        "\\end{table}"
    )
    return latex_code

In [16]:
latex_output = generate_combined_table(cvg_case2, mae_case2)
with open(f"output/tex_{mode}_{gene_process}.tex", "w") as f:
    f.write(latex_output)